In [2]:
import cv2
import mediapipe as mp
import time
import numpy as np
import import_ipynb
import HandTrackingModule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

######################
wCam, hCam = 640, 480
######################

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector = htm.handDetector(detectionCon=0.7)
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volRange = volume.GetVolumeRange()
volMin = volRange[0]
volMax = volRange[1]
vol = 0
volBar = 400
volPer = 0
adjustVolume = True  # State to control volume adjustment

while True:
    success, img = cap.read()
    if not success:
        print("Error: Could not read video")
        break
        
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        # Thumb and index finger tips
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        
        cv2.circle(img, (x1, y1), 15, (255, 0, 0), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 0), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
        cv2.circle(img, (cx, cy), 15, (255, 0, 0), cv2.FILLED)
        
        # Determine thumb position relative to the index finger
        if y1 < y2:
            # Thumb up - lock volume
            adjustVolume = False
        else:
            # Thumb down - enable volume adjustment
            adjustVolume = True
        
        if adjustVolume:
            length = math.hypot(x2 - x1, y2 - y1)
            
            # Interpolating length to volume level
            vol = np.interp(length, [22, 120], [volMin, volMax])
            volBar = np.interp(length, [22, 120], [400, 150])
            volPer = np.interp(length, [22, 120], [0, 100])
            
            volume.SetMasterVolumeLevel(vol, None)
            
    # Draw volume bar
    cv2.rectangle(img, (50, 150), (85, 400), (0, 255, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (0, 255, 0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)}%', (40, 450), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    cv2.putText(img, f'FPS: {int(fps)}', (10, 70), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
    
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
